## Part 2:  Application
# Combat: GAN vs. WGAN

In [69]:
import os
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tensorflow.examples.tutorials.mnist import input_data


In [16]:
mnist = input_data.read_data_sets('../data/MNIST_data', one_hot=True)

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


In [24]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [49]:
mb_size = 32
X_dim = 784
Z_dim = 10
h_dim = 128

In [75]:
# Discriminator Net
X = tf.placeholder(tf.float32, shape=[None, X_dim])

D_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

D_W2 = tf.Variable(xavier_init([h_dim, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

# Generator Net
Z = tf.placeholder(tf.float32, shape=[None, Z_dim])

G_W1 = tf.Variable(xavier_init([Z_dim, h_dim]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

## Vanilla GAN

In [76]:
def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob


def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit

In [ ]:
G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
G_loss = -tf.reduce_mean(tf.log(D_fake))


D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('../data/vgan-out/'):
    os.makedirs('../data/vgan-out/')

i = 0
vgan_train_log = []
for it in range(1000000):
    start = time.time()
    X_mb, _ = mnist.train.next_batch(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})
        end = time.time()
        
        fig = plot(samples)
        plt.savefig('../data/vgan-out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)
        
        vgan_train_log.append((it, D_loss_curr, G_loss_curr))
        
        print('Iter: {}; D_loss: {:.4}; G_loss: {:.4}: Time: {} s'
              .format(it+1000, D_loss_curr, G_loss_curr, t))        


Iter: 1000; D_loss: 1.297; G_loss: 2.468: Time: 0.0195109844208 s
Iter: 2000; D_loss: 0.003681; G_loss: 6.906: Time: 0.0195109844208 s
Iter: 3000; D_loss: 0.06289; G_loss: 5.819: Time: 0.0195109844208 s
Iter: 4000; D_loss: 0.2845; G_loss: 2.851: Time: 0.0195109844208 s
Iter: 5000; D_loss: 0.2824; G_loss: 4.038: Time: 0.0195109844208 s
Iter: 6000; D_loss: 0.3532; G_loss: 3.204: Time: 0.0195109844208 s
Iter: 7000; D_loss: 0.4775; G_loss: 3.686: Time: 0.0195109844208 s
Iter: 8000; D_loss: 0.5516; G_loss: 2.718: Time: 0.0195109844208 s
Iter: 9000; D_loss: 0.8966; G_loss: 3.127: Time: 0.0195109844208 s
Iter: 10000; D_loss: 0.715; G_loss: 2.096: Time: 0.0195109844208 s
Iter: 11000; D_loss: 0.7747; G_loss: 2.883: Time: 0.0195109844208 s
Iter: 12000; D_loss: 0.5343; G_loss: 2.212: Time: 0.0195109844208 s
Iter: 13000; D_loss: 0.6591; G_loss: 2.48: Time: 0.0195109844208 s
Iter: 14000; D_loss: 0.8519; G_loss: 2.121: Time: 0.0195109844208 s
Iter: 15000; D_loss: 0.4634; G_loss: 2.416: Time: 0.01951

In [ ]:
vgan_log = open('vgan_train_log.txt', 'w')
for item in vgan_train_log:
    vgan_log.write(str(item))

## GAN with cross entropy as  D_loss

In [77]:
G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

# Alternative losses: cross entropy between D’s 
# -------------------
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, 
                                                                     labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, 
                                                                     labels=tf.zeros_like(D_logit_fake)))

D_loss = D_loss_real + D_loss_fake

G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, 
                                                                labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('../data/gan-out/'):
    os.makedirs('../data/gan-out/')

i = 0
gan_train_log = []
for it in range(1000000):
    start = time.time()
    X_mb, _ = mnist.train.next_batch(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})
        end = time.time()
        
        fig = plot(samples)
        plt.savefig('../data/gan-out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)
        
        gan_train_log.append((it, D_loss_curr, G_loss_curr))
        
        print('Iter: {}; D_loss: {:.4}; G_loss: {:.4}: Time: {} s'
              .format(it+1000, D_loss_curr, G_loss_curr, t))        


Iter: 1000; D_loss: 0.8983; G_loss: 2.374: Time: 0.0195109844208 s
Iter: 2000; D_loss: 0.02112; G_loss: 7.411: Time: 0.0195109844208 s
Iter: 3000; D_loss: 0.07241; G_loss: 4.28: Time: 0.0195109844208 s
Iter: 4000; D_loss: 0.1601; G_loss: 4.833: Time: 0.0195109844208 s
Iter: 5000; D_loss: 0.4607; G_loss: 4.435: Time: 0.0195109844208 s
Iter: 6000; D_loss: 0.3044; G_loss: 3.797: Time: 0.0195109844208 s
Iter: 7000; D_loss: 0.2695; G_loss: 4.776: Time: 0.0195109844208 s
Iter: 8000; D_loss: 0.5058; G_loss: 2.599: Time: 0.0195109844208 s
Iter: 9000; D_loss: 0.6018; G_loss: 3.267: Time: 0.0195109844208 s
Iter: 10000; D_loss: 0.3137; G_loss: 3.113: Time: 0.0195109844208 s
Iter: 11000; D_loss: 0.3935; G_loss: 3.373: Time: 0.0195109844208 s
Iter: 12000; D_loss: 0.3213; G_loss: 2.267: Time: 0.0195109844208 s
Iter: 13000; D_loss: 0.5578; G_loss: 2.239: Time: 0.0195109844208 s
Iter: 14000; D_loss: 0.6034; G_loss: 1.845: Time: 0.0195109844208 s
Iter: 15000; D_loss: 0.4502; G_loss: 1.872: Time: 0.0195

In [84]:
gan_log = open('gan_train_log.txt', 'w')
for item in gan_train_log:
    gan_log.write(str(item))

## WGAN

In [51]:
def w_generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob


def w_discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    out = tf.matmul(D_h1, D_W2) + D_b2
    return out

In [74]:
G_sample = w_generator(Z)
D_real = w_discriminator(X)
D_fake = w_discriminator(G_sample)

D_loss = tf.reduce_mean(D_real) - tf.reduce_mean(D_fake)
G_loss = -tf.reduce_mean(D_fake)

D_solver = (tf.train.RMSPropOptimizer(learning_rate=1e-4/2)
            .minimize(-D_loss, var_list=theta_D))
G_solver = (tf.train.RMSPropOptimizer(learning_rate=1e-4/2)
            .minimize(G_loss, var_list=theta_G))

clip_D = [p.assign(tf.clip_by_value(p, -0.01, 0.01)) for p in theta_D]

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('../data/wgan-out/'):
    os.makedirs('../data/wgan-out/')

i = 0
wgan_train_log = []
for it in range(1000000):
    start = time.time()
    # Train D more than G
    for _ in range(5):
        X_mb, _ = mnist.train.next_batch(mb_size)

        _, D_loss_curr, _ = sess.run(
            [D_solver, D_loss, clip_D],
            feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)}
        )

    _, G_loss_curr = sess.run(
        [G_solver, G_loss],
        feed_dict={Z: sample_Z(mb_size, Z_dim)}
    )

    if it % 1000 == 0:
        
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})
        end = time.time()
        t = end - start
        fig = plot(samples)
        plt.savefig('../data/wgan-out/{}.png'
                    .format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)
        
        wgan_train_log.append((it, D_loss_curr, G_loss_curr))
        
        print('Iter: {}; D_loss: {:.4}; G_loss: {:.4}: Time: {} s'
              .format(it+1000, D_loss_curr, G_loss_curr, t))        

Iter: 1000; D_loss: -0.01517; G_loss: -0.02339: Time: 0.131274938583 s
Iter: 2000; D_loss: 0.7509; G_loss: 0.5174: Time: 0.0185010433197 s
Iter: 3000; D_loss: 0.06148; G_loss: 0.04872: Time: 0.0181849002838 s
Iter: 4000; D_loss: 0.007315; G_loss: -0.03029: Time: 0.0182549953461 s
Iter: 5000; D_loss: -0.0007318; G_loss: -0.002305: Time: 0.0181109905243 s
Iter: 6000; D_loss: 0.003452; G_loss: 0.03762: Time: 0.0183629989624 s
Iter: 7000; D_loss: -0.002478; G_loss: -0.0485: Time: 0.0185549259186 s
Iter: 8000; D_loss: 0.001471; G_loss: 0.004954: Time: 0.0185699462891 s
Iter: 9000; D_loss: 0.004065; G_loss: 0.01647: Time: 0.0184619426727 s
Iter: 10000; D_loss: 0.003906; G_loss: -0.009036: Time: 0.0183811187744 s
Iter: 11000; D_loss: 0.04819; G_loss: 0.05082: Time: 0.0186378955841 s
Iter: 12000; D_loss: 0.07299; G_loss: -0.06687: Time: 0.0183000564575 s
Iter: 13000; D_loss: 0.04786; G_loss: -0.005797: Time: 0.018424987793 s
Iter: 14000; D_loss: 0.0409; G_loss: -0.005913: Time: 0.0184240341187

In [83]:
wgan_log = open('wgan_train_log.txt', 'w')
for item in wgan_train_log:
    wgan_log.write(str(item))